<h2><b>Importing All Required Libraries</b></h2>

In [1]:
import re
import os
import csv
import nltk
import nlp
import wordninja
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Artemis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Artemis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<h2><b>Import Datasets</b></h2>

In [35]:
df = pd.read_csv("ds/t4sa-with-text-and-labels.csv")
df.head()

,TWID,NEG,NEU,POS,TWEET,SENTIMENT
0,768096868504969216,0.049398,0.861395,0.089207,#Incredible #India #Atulya #Bharat - Land of S...,neutral
1,768097619281227776,0.006598,0.046810,0.946591,RT @KendallHuntRPD: The #firstdayofschool for ...,positive
2,768097627695042560,0.006389,0.018663,0.974948,"RT @PEPalerts: This September, @YESmag is taki...",positive
3,768097640269643776,0.013075,0.110913,0.876012,RT @BantySrkian: #SRK and kajol in the making ...,positive
4,768097640278089729,0.004939,0.029469,0.965591,RT @MianUsmanJaved: Congratulations Pakistan o...,positive


In [36]:
del df['TWID']
df.head()

,NEG,NEU,POS,TWEET,SENTIMENT
0,0.049398,0.861395,0.089207,#Incredible #India #Atulya #Bharat - Land of S...,neutral
1,0.006598,0.046810,0.946591,RT @KendallHuntRPD: The #firstdayofschool for ...,positive
2,0.006389,0.018663,0.974948,"RT @PEPalerts: This September, @YESmag is taki...",positive
3,0.013075,0.110913,0.876012,RT @BantySrkian: #SRK and kajol in the making ...,positive
4,0.004939,0.029469,0.965591,RT @MianUsmanJaved: Congratulations Pakistan o...,positive


<h2><b>Viterbi Algorithm</b></h2>

In [37]:
def viterbi_segment(text):
    text.lower()
    probs, lasts = [1.0], [0]
    for i in range(1, len(text) + 1):
        prob_k, k = max((probs[j] * word_prob(text[j:i]), j)
                        for j in range(max(0, i - max_word_length), i))
        probs.append(prob_k)
        lasts.append(k)
    words = []
    i = len(text)
    while 0 < i:
        words.append(text[lasts[i]:i])
        i = lasts[i]
    words.reverse()
    return words

def word_prob(word): return dictionary[word] / total
def words(text): return re.findall('[a-z]+', text.lower()) 
dictionary = Counter(words(open('ds/big.txt').read()))
max_word_length = max(map(len, dictionary))
total = float(sum(dictionary.values()))

In [38]:
viterbi_segment("helloworld")

['hello', 'world']

<h2><b>Make Functions That Will Perform Preprocessing</b></h2>

In [52]:
def unjoin_words(text):
    lst = text.split()
    sent = []
    for x in lst:
        sent.append(" ".join(viterbi_segment(x)))
    return " ".join(sent)

In [53]:
unjoin_words("hellohoware you")

'hello how are you'

In [8]:
# extract all hashtags
def hashtags(text):
    hash = re.findall(r'#(\w+)', text)
    return hash

In [9]:
# removing tweeted at '@username'
def remove_users(tweet) :
    user = re.sub('@[A-Za-z]+[A-Za-z0-9-_]+', '', tweet)
    return user

In [10]:
def remove_links(tweet):
    url = re.sub(r"http\S+", "", tweet)
    return(url)

In [11]:
# do not use this function if you are doing translation work
def non_ascii(s):
    return "".join(i for i in s if ord(i)<128)

In [12]:
# switch all words to lowercase
def lower(text):
    return text.lower()

In [13]:
# removes all stopwords
def remove_stopwords(tweet):
    stops = set(stopwords.words("english"))
    # add custom words
#     stops.update(('mailto', 'brb', 'it', 'this', 'yolo'))
    new_tweet = ' '.join([word for word in tweet.split() if word not in stops])
    return new_tweet

In [14]:
# remove email_ids
def email_address(tweet):
    email = re.compile(r'[\w\.-]+@[\w\.-]+com')
    return email.sub(r'', tweet)

In [15]:
# remove all kind of punctuations
def punct(text):
    token = RegexpTokenizer(r'\w+')
    text = token.tokenize(text)
    text = ' '.join(text)
    return text

In [16]:
# remove all numbers
def remove_digits(text):
    pattern = r'[^a-zA-Z.,!?/:;\"\'\s]'
    return re.sub(pattern, '', text)

In [17]:
# remove all special characters
def remove_special_characters(text):
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    return re.sub(pat, '', text)

In [18]:
# remove all punctuation
def punct(text):
    token = RegexpTokenizer(r'\w+')
    text = token.tokenize(text)
    text = ' '.join(text)
    return text

In [19]:
# remove RT
def remove_rt(text):
    return re.sub('RT :', '', text)

<h2><b>Negation</b></h2>

In [20]:
# dictionary for negation
givenpatterns = [(r'won\'t', 'will not'), (r'don\'t', 'do not'),
				 (r'can\'t', 'can not'), (r'\'s', 'is'),
				 (r'wouldn\'t', 'would not'), (r'doesn\'t', 'does not'),
				 (r'isn\'t', 'is not'), (r'shan\'t', 'shall not'),
				 (r'aren\'t', 'are not'), (r'hasn\'t', 'has not'),
				 (r'wasn\'t', 'was not'), (r'weren\'t', 'were not'),
				 (r'haven\'t', 'have not'), (r'hasn\'t', 'has not'),
				 (r'hadn\'t', 'had not'), (r'didn\'t', 'did not'),
				 (r'couldn\'t', 'could not'), (r'shouldn\'t', 'should not'),
				 (r'mightn\'t', 'might not'), (r'mustn\'t', 'must not'),
				 (r'wont', 'will not'), (r'dont', 'do not'), (r"shalln't", 'shall not'),
				 (r'cant', 'can not'), (r'wouldnt', 'would not'),
				 (r'doesnt', 'does not'), (r'isnt', 'is not'),
				 (r'shant', 'shall not'), (r'arent', 'are not'),
				 (r'hasnt', 'has not'), (r'wasnt', 'was not'),
				 (r'werent', 'were not'), (r'havent', 'have not'),
				 (r'hasnt', 'has not'), (r'hadnt', 'had not'),
				 (r'didnt', 'did not'), (r'couldnt', 'could not'),
				 (r'shouldnt', 'should not'), (r'mightnt', 'might not'),
				 (r'mustnt', 'must not') ]

In [21]:
class AntonymReplacer(object):
	def replace(self, word): #This function is used to find the antonym 
		antonyms = set()
		for syn in wordnet.synsets(word):
			for lemma in syn.lemmas():
				for antonym in lemma.antonyms():
					antonyms.add(antonym.name())
		if len(antonyms)== 1:
			return antonyms.pop()
		else:
			return None


	def negreplace(self, string):# This function is used to evaluate the sentence and find the negation
		i=0
		sent= word_tokenize(string)
		len_sent = len(sent)
		
		words= []
		fsent = " "
		while i<len_sent:
			word=sent[i]
			
			if word == 'not' and i+1 < len_sent:
				ant = self.replace(sent[i+1])
				if ant:
					words.append(ant)
					fsent += ant + " "
					i = i + 2
					continue
			words.append(word)
			fsent += word + " "
			i = i + 1

		return fsent
    
class RegexReplacer(object): # This is a regular expression function to repalce the words like don't with do not
	def __init__(self):
		self.patterns = givenpatterns

	def replace(self,text):
		for(raw,rep) in self.patterns:
			regex = re.compile(raw, re.IGNORECASE)
			text = regex.sub(rep, text)
		return text 

In [22]:
rep=AntonymReplacer()
reg=RegexReplacer()

In [23]:
def apply_negation(text):
    jword = reg.replace(text)
    final = rep.negreplace(jword)
    return final

In [24]:
apply_negation("I shouldn't shalln't that")

' I should not shall not that '

<h2><b>Apply Preprocessing</b></h2>

In [25]:
df['HASHTAGS'] = df.TWEET.apply(func = hashtags)
df['TEXT'] = df.TWEET.apply(func = remove_users)
df['TEXT'] = df.TEXT.apply(func = remove_links)
df['TEXT'] = df.TEXT.apply(func = remove_rt)
df['TEXT'] = df.TEXT.apply(func = non_ascii)
df['TEXT'] = df.TEXT.apply(func = remove_stopwords)
df['TEXT'] = df.TEXT.apply(func = remove_digits)
df['TEXT'] = df.TEXT.apply(func = remove_special_characters)
df['TEXT'] = df.TEXT.apply(func = punct)
df['TEXT'] = df.TEXT.apply(func = email_address)
# df['TEXT'] = df.TEXT.apply(func = unjoin_words)
df['TEXT'] = df.TEXT.apply(func = lower)
df.head()

,TWEET,SENTIMENT,HASHTAGS,TEXT
0,#Incredible #India #Atulya #Bharat - Land of S...,neutral,"[Incredible, India, Atulya, Bharat, BeProud]",incredible india atulya bharat land seekers be...
1,RT @KendallHuntRPD: The #firstdayofschool for ...,positive,"[firstdayofschool, EducationMatters]",the firstdayofschool students amp teachers goo...
2,"RT @PEPalerts: This September, @YESmag is taki...",positive,[],this september taking maine mendozas surprise ...
3,RT @BantySrkian: #SRK and kajol in the making ...,positive,[SRK],srk kajol making ddlj song both greatest ever
4,RT @MianUsmanJaved: Congratulations Pakistan o...,positive,"[No1TestTeam, JI_PakZindabadRallies]",congratulations pakistan becoming notestteam w...


In [29]:
df.to_csv("ds/t4sa-preprocessed.csv")

In [30]:
df.describe()

,TWEET,SENTIMENT,HASHTAGS,TEXT
count,1736,1736,1736,1736
unique,1585,3,611,1524
top,RT @allkpop: GOT7 look back on 115 days of 'Fl...,neutral,[],got look back days fly tour special mv
freq,20,1155,977,20


In [56]:
df['SENTIMENT'].value_counts()

neutral     1155
positive     531
negative      50
Name: SENTIMENT, dtype: int64